In [29]:
import pandas as pd
from pathlib import Path
import scipy
parks_data = pd.read_csv(Path.home() / 'Downloads' / 'poi_with_population_count_2022-01-31.csv')
hps = pd.read_csv(Path.home() / 'Downloads' / '2022-01-26-home_panel_summary.csv')
census = pd.read_csv(Path.home() / 'Downloads' / 'cbg_b01.csv')

In [8]:
print(hps.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 271398 entries, 0 to 271397
Data columns (total 7 columns):
 #   Column                          Non-Null Count   Dtype 
---  ------                          --------------   ----- 
 0   date_range_start                271398 non-null  object
 1   date_range_end                  271398 non-null  object
 2   region                          271398 non-null  object
 3   iso_country_code                271398 non-null  object
 4   census_block_group              271398 non-null  object
 5   number_devices_residing         271398 non-null  int64 
 6   number_devices_primary_daytime  271398 non-null  int64 
dtypes: int64(2), object(5)
memory usage: 14.5+ MB
None


In [6]:
print(parks_data.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2637 entries, 0 to 2636
Data columns (total 37 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   Unnamed: 0                 2637 non-null   int64  
 1   placekey                   2637 non-null   object 
 2   location_name_x            2637 non-null   object 
 3   latitude                   2637 non-null   float64
 4   longitude                  2637 non-null   float64
 5   Unnamed: 0.1               2337 non-null   float64
 6   parent_placekey            343 non-null    object 
 7   location_name_y            2337 non-null   object 
 8   street_address             2337 non-null   object 
 9   city                       2337 non-null   object 
 10  region                     2337 non-null   object 
 11  postal_code                2337 non-null   float64
 12  iso_country_code           2337 non-null   object 
 13  safegraph_brand_ids        26 non-null     objec

In [15]:
print(census.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 220333 entries, 0 to 220332
Columns: 161 entries, census_block_group to B01003m1
dtypes: float64(60), int64(101)
memory usage: 270.6 MB
None


In [23]:
census_first = census[['census_block_group', 'B01001e1']]
census_first = census_first.loc[census_first['census_block_group'].astype(str).str.startswith('36')]
print(census_first.info())

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15463 entries, 127690 to 143152
Data columns (total 2 columns):
 #   Column              Non-Null Count  Dtype
---  ------              --------------  -----
 0   census_block_group  15463 non-null  int64
 1   B01001e1            15463 non-null  int64
dtypes: int64(2)
memory usage: 362.4 KB
None


In [27]:
pop_nys = census_first['B01001e1'].sum()
print(pop_nys)

19572319


In [25]:
#find state devices

hps_ny = hps.loc[hps['region'] == 'ny']
devices_nys = hps_ny['number_devices_residing'].sum()
print(devices_nys)

852188


In [28]:
#find statewide multiplier

mult = pop_nys / devices_nys
print(mult)

22.967137533032616


In [34]:
compare_df = pd.DataFrame()
compare_df['raw_counts']= parks_data['raw_visitor_counts']
compare_df['state_normalized'] = compare_df['raw_counts'] * mult
compare_df['cbg_normalized'] = parks_data['visits_pop_calc']

In [38]:
compare_df['error'] = (compare_df['cbg_normalized'] - compare_df['state_normalized']).apply(abs)
compare_df['percent_error'] = (compare_df['cbg_normalized'] - compare_df['state_normalized']).apply(abs) / compare_df['cbg_normalized']
print(f"mean error: {compare_df['error'].mean()}")
print(f"mean percent error: {compare_df['percent_error'].mean()}")
print(f"avg state normalized: {compare_df['state_normalized'].mean()}")
print(f"avg cbg normalized: {compare_df['cbg_normalized']}")


mean error: 1958.8608511341095
mean percent error: 0.5956371868427998
avg state normalized: 2808.870109912794
avg cbg normalized: 0       3607.309091
1       1058.234375
2       6290.619779
3       3636.989916
4        415.991006
           ...     
2632    1969.333333
2633     862.898180
2634    2092.149859
2635     365.867769
2636            NaN
Name: cbg_normalized, Length: 2637, dtype: float64
